In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version
# from pyspark import SparkContext, SparkConf

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

# conf = SparkConf().set('spark.executor.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
#  set('spark.driver.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
#  setAppName('pyspark_aws').setMaster('local[*]')
# sc=SparkContext(conf=conf)
# sc.setSystemProperty('com.amazonaws.services.s3.enableV4', 'true')


Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,136 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,313 kB]
Get:14 http:

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-16 20:30:43--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.67MB/s    in 0.5s    

2023-03-16 20:30:44 (1.67 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_download_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
video_download_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [ ]:
# Get the number of rows in the DataFrame.
video_download_df.count()

4057147

# Transform the Data

## Create the "review_id_table".

In [ ]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df=video_download_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3FU16928EP5TC|   12190288|B00AYB1482|     668895143| 2015-08-31|
|R1IZHHS1MH3AQ4|   30549954|B00KQD28OM|     246219280| 2015-08-31|
| R52R85WC6TIAH|   52895410|B01489L5LQ|     534732318| 2015-08-31|
| R7HOOYTVIB0DS|   27072354|B008LOVIIK|     239012694| 2015-08-31|
|R1XQ2N5CDOZGNX|   26939022|B0094LZMT0|     535858974| 2015-08-31|
|R1HCST57W334KN|    4772040|B0112OSOQE|      38517795| 2015-08-31|
|R32BUTYQS1ZJBQ|   12910040|B000NPE5SA|     373323715| 2015-08-31|
|  RH4SXPL4L9QU|   38805573|B00XWV4QXG|     633842417| 2015-08-31|
|R37INWIQA5YW8N|   37100714|B00X8UKOUK|     666093513| 2015-08-31|
|R18GSVAAS3N8GX|   41234409|B00OOKXTFU|     801680808| 2015-08-31|
|R2QCG0ZZOHYUOX|   24698954|B00URDJ600|     174981485| 2015-08-31|
|R2B1OCYMUVMS5C|   11956809|B009JUZ5I4|     654293980| 2015-08

## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = video_download_df.dropDuplicates(['product_id','product_title'])
products_df.count()

166748

## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
 
customers_df = review_id_df.groupBy("customer_id").count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   36427874|    1|
|   17687096|   10|
|   43798350|    4|
|   49387236|    1|
|   24321836|    2|
|   42543176|    1|
|   16298288|    4|
|   12333316|    1|
|   25790304|    1|
|   13786319|    1|
|   24796052|    1|
|   53049917|    4|
|   30478898|    1|
|   20829560|    3|
|   29699080|    3|
|    8681732|    3|
|   34220092|    2|
|   13727857|   14|
|   13480593|    3|
|   49935991|    1|
+-----------+-----+
only showing top 20 rows



## Create the "vine_table".

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df=video_download_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3FU16928EP5TC|          5|            0|          0|   N|
|R1IZHHS1MH3AQ4|          5|            0|          0|   N|
| R52R85WC6TIAH|          4|           17|         18|   N|
| R7HOOYTVIB0DS|          5|            0|          0|   N|
|R1XQ2N5CDOZGNX|          5|            0|          0|   N|
|R1HCST57W334KN|          5|            5|          6|   N|
|R32BUTYQS1ZJBQ|          4|            1|          1|   N|
|  RH4SXPL4L9QU|          3|            0|          0|   N|
|R37INWIQA5YW8N|          2|            0|          0|   N|
|R18GSVAAS3N8GX|          3|            0|          0|   N|
|R2QCG0ZZOHYUOX|          5|            0|          0|   N|
|R2B1OCYMUVMS5C|          5|            0|          0|   N|
|R131ZJWHK0XXY0|          4|            0|          0|   N|
| RY6RIUNL03N8M|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "xxxxxxxxxxxx", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


In [ ]:
# Write products_df to table in RDS
product_df.write.jdbc(url=jdbc_url, table='product_table', mode=mode, properties=config)


In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)


In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)


In [ ]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
software_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
software_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [ ]:
# Get the number of rows in the DataFrame.
software_df.count()

341931

In [ ]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
software_review_id_df=software_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")
software_review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3EFW2STIYIY0I|   42605767|B00MUTIDKI|     248732228| 2015-08-31|
|R12NR0R5A9F7FT|   51771800|B00EPACNUG|     531462352| 2015-08-31|
|R1LSH74R9XAP59|   16053526|B00164AZA4|     473982505| 2015-08-31|
|R1QXUNTF76K7L6|   15319481|B00E6LIEFM|     189774198| 2015-08-31|
|R2F7DR75PS8NKT|    1441820|B00VWEBG06|     852470365| 2015-08-31|
|R2C1DJSCC8UFS6|   37107850|B00EP7AP7C|     279360628| 2015-08-31|
|R1AXGS1W4YFXMX|     302120|B00OW2PET4|     729971168| 2015-08-31|
|R1XU1B93402SYJ|   20193077|B00N4OLCRO|     776572654| 2015-08-31|
|R2U432NB3OPVR0|   13106017|B005CELN8W|     222071424| 2015-08-31|
|R3R6FIMIOQ5SP9|   32587108|B005CELL1G|     168801430| 2015-08-31|
|R12TX6V09C9QNQ|   32020017|B00H09BOXQ|     455502995| 2015-08-31|
|R33UCII6YKUMKV|   13376158|B00MCLGAAO|     652069521| 2015-08

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
sw_products_df = software_df.dropDuplicates(['product_id','product_title'])
sw_products_df.count()

28736

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
 
sw_customers_df = software_review_id_df.groupBy("customer_id").count()
sw_customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   15634680|    1|
|   44696507|    1|
|   19901367|    1|
|   11337682|    1|
|   52021773|    1|
|   29916198|    1|
|   42030944|    1|
|   52765209|    1|
|   13519509|    2|
|   39320350|    5|
|   23249582|    1|
|   46351928|    3|
|   44179620|    1|
|     637252|    1|
|   34038793|    1|
|   51136490|    1|
|   49354815|    1|
|     149761|    2|
|   43333798|    1|
|    4657027|    1|
+-----------+-----+
only showing top 20 rows



In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
sw_vine_df=software_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
sw_vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3EFW2STIYIY0I|          1|            2|          2|   N|
|R12NR0R5A9F7FT|          5|            0|          0|   N|
|R1LSH74R9XAP59|          2|            0|          1|   N|
|R1QXUNTF76K7L6|          2|            0|          0|   N|
|R2F7DR75PS8NKT|          5|            0|          0|   N|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|
|R1XU1B93402SYJ|          1|            1|          1|   N|
|R2U432NB3OPVR0|          5|            0|          0|   N|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|
|R12TX6V09C9QNQ|          5|            0|          0|   N|
|R33UCII6YKUMKV|          3|            2|          2|   N|
| RZKDAB9TGO053|          1|            0|          0|   N|
|R2EMN2EEDN73ZA|          4|            

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "xxxxxxxxxxxx", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
software_review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


In [ ]:
# Write products_df to table in RDS
sw_product_df.write.jdbc(url=jdbc_url, table='product_table', mode=mode, properties=config)


In [ ]:
# Write customers_df to table in RDS
sw_customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)


In [ ]:
# Write vine_df to table in RDS
sw_vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
